<a href="https://colab.research.google.com/github/gazon1/Recco-challenge/blob/master/%D1%80%D0%B0%D0%B1%D0%BE%D1%87%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from sklearn.metrics import # Rekko challenge 2019

```
                           /$$$$$$$  /$$$$$$$$ /$$   /$$ /$$   /$$  /$$$$$$ 
                          | $$__  $$| $$_____/| $$  /$$/| $$  /$$/ /$$__  $$
                          | $$  \ $$| $$      | $$ /$$/ | $$ /$$/ | $$  \ $$
                          | $$$$$$$/| $$$$$   | $$$$$/  | $$$$$/  | $$  | $$
                          | $$__  $$| $$__/   | $$  $$  | $$  $$  | $$  | $$
                          | $$  \ $$| $$      | $$\  $$ | $$\  $$ | $$  | $$
                          | $$  | $$| $$$$$$$$| $$ \  $$| $$ \  $$|  $$$$$$/
                          |__/  |__/|________/|__/  \__/|__/  \__/ \______/ 
                                                                            
```

Добро пожаловать на соревнование по машинному обучению от онлайн-кинотеатра [Okko](http://okko.tv) Rekko Challenge 2019.

В этом ноутбуке мы покажем вам пример простого но полного решения, от загрузки данных до формирования ответа. Для работы нам понадобятся библиотеки `pandas`, `numpy`, `scipy`, `implicit`, `pprint`, `tqdm`. Установить их в вашем рабочем окружении можно следующей командой.
```
pip install pandas numpy scipy implicit pprint tqdm
```

In [0]:
import os
import json
import pandas as pd
import numpy as np


import tqdm
import scipy.sparse as sp
from sklearn.metrics import average_precision_score as score
import time
from matplotlib import pyplot as plt

import itertools
from pprint import pprint

DATA_PATH = ''
%matplotlib inline

# For Google Colab only:
import sys
sys.path.append('/content/gdrive/My Drive/okko competition')
#from reco_utils.recommender.sar.sar_singlenode import SARSingleNode
from google.colab import drive
drive.mount('/content/gdrive')

DATA_PATH = 'gdrive/My Drive/okko competition/data'
CODE_PATH = 'gdrive/My Drive/okko competition/code'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Hyperparametrs:

WATCHED_TIME = 600 # выкидываем передачи, просмотренные меньше WATCHED_TIME. ЧТобы ничего не выкидывать - ставим 0
MIN_RATING = 3 # ниже этого рейтинга все выкидываем


In [0]:
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, Dense,LSTM, CuDNNGRU#, CuDNNLSTM, CuDNNGRU
from keras.models import Model

import keras
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, BatchNormalization


# Let's set random seed
import tensorflow as tf
tf.set_random_seed(42)
np.random.seed(42)

### Загрузка данных

`catalogue.json` содержит анонимизированную метаинформацию о доступных в сервисе фильмах и сериалах.

In [0]:
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}
catalogue = pd.DataFrame.from_dict(catalogue,'index').reset_index()
# av_hot - нужно для валидации. Эта колонка равна 1, если фильм или сериал может быть потребленным
catalogue['av_hot'] = catalogue['availability'].apply(lambda x: 1 if x == [] or 'subscription' in x else 0)
catalogue = catalogue.rename(index=str, columns={'index':'element_uid'})

 - `attributes` — мешок атрибутов
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`)
 - `duration` — длительность в минутах, округлённая до десятков (продолжительность серии для сериалов и многосерийных фильмов)
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков
 - `type` — принимает значения `movie`, `multipart_movie` или `series`

`test_users.json` содержит список пользователей, для которых необходимо построить предсказание

In [0]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])
len(test_users)

50000

`transactions.csv` — список всех транзакций за определённый период времени

In [0]:
%%time
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

CPU times: user 6.89 s, sys: 112 ms, total: 7.01 s
Wall time: 7.26 s


In [0]:
%%time
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

CPU times: user 199 ms, sys: 9.83 ms, total: 209 ms
Wall time: 224 ms


In [0]:
%%time
bookmarks = pd.read_csv(
    os.path.join(DATA_PATH, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }
)

CPU times: user 368 ms, sys: 13.7 ms, total: 381 ms
Wall time: 397 ms


In [0]:
bookmarks['rating'] = 1
_bookmarks = bookmarks[['rating', 'user_uid', 'element_uid','ts']]

_ratings = ratings[ratings['rating']>5]
_ratings['rating'] = 1

_transaction = transactions[['element_uid', 'user_uid','ts','watched_time']]
_transaction = _transaction[(_transaction['watched_time']>700)]
_transaction['rating'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [0]:
all_ratings = pd.concat([ _transaction[['user_uid', 'element_uid', 'rating', 'ts']],_bookmarks[['user_uid', 'element_uid', 'rating', 'ts']]])
all_ratings = pd.concat([all_ratings, _ratings[['user_uid', 'element_uid', 'rating','ts']]])
# all_ratings = all_ratings.drop_duplicates(['user_uid','element_uid'],keep='first')
all_ratings = all_ratings.sort_values(by=['ts'])
all_ratings= all_ratings.rename(index=str,columns={'element_uid':'itemID','user_uid':'userID','ts':'timestamp'})

In [0]:
#надо найти оптимальный размер, который взлезет в память. в 12 ГБ РАМ влезает около 800к записей. Всего их 11 млн
all_ratings.head(3)

,userID,itemID,rating,timestamp
9643011,283774,6189,1,4.173063e+07
9643010,59148,6099,1,4.173063e+07
9643009,50431,8888,1,4.173063e+07


In [0]:
# temp = all_ratings.groupby('userID').agg({'itemID':'count'}).reset_index()
# users = temp[temp['itemID'] >= 10]['userID'].values
# all_ratings = all_ratings[all_ratings['userID'].isin(users)]


In [0]:
all_ratings.itemID = all_ratings.itemID.astype('category')
all_ratings['categ_id'] = all_ratings.itemID.cat.codes + 1
cat_to_element_uid = dict(zip(
    range(1, len(all_ratings.itemID.cat.categories) + 1),
    all_ratings.itemID.cat.categories
))



In [0]:
%%time
import tqdm
tqdm.tqdm.pandas()
sequences = all_ratings.groupby('userID')['categ_id'].progress_apply(list)

100%|██████████| 496046/496046 [00:29<00:00, 16739.70it/s]

CPU times: user 30.1 s, sys: 73.9 ms, total: 30.2 s
Wall time: 30.2 s


In [0]:
sequences2use = sequences[sequences.apply(len) >= 10]

In [0]:
maxlen = 10 # Length of sequences in X
X = []
y = []

def slice_sequence(seq, num_slices):
    for i in range(1, num_slices):
        X.append(seq[-(i+maxlen): -i])
        y.append(seq[-i])
        
for seq in tqdm.tqdm(sequences2use):
  if len(seq) == 11:
    slice_sequence(seq, 2)
#   if len(seq) == 12:
#     slice_sequence(seq, 2)
#   if len(seq) == 13:
#     slice_sequence(seq, 2)
#   if len(seq) == 14:
#     slice_sequence(seq, 2)
#   if len(seq) == 15:
#     slice_sequence(seq, 2)
#   if len(seq) == 16:
#     slice_sequence(seq, 2)
#   if len(seq) >17:
#     slice_sequence(seq, len(seq)-maxlen)
  if len(seq) >11:
    slice_sequence(seq, len(seq)-maxlen)

100%|██████████| 228844/228844 [00:09<00:00, 24097.33it/s]


In [0]:
lens = [len(x) for x in X]
max(lens), min(lens), np.mean(lens), np.median(lens)

(10, 10, 10.0, 10.0)

In [0]:
from keras.preprocessing.sequence import pad_sequences

# We should pad our sequences with 0 values, so they all will have the same length
X = pad_sequences(X, maxlen=maxlen)
y = np.array(y)
X.shape, y.shape

((5100518, 10), (5100518,))

In [0]:
def save_model(model, name, prefix=''):
    model_json = model.to_json()
    with open(os.path.join(CODE_PATH, prefix + "model.json"), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(os.path.join(CODE_PATH, prefix + f"{name}.h5"))
    print("Saved model to disk")

In [0]:
from keras.models import model_from_json
def load_model():
    json_file = open(os.path.join(CODE_PATH, 'model.json'), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(os.path.join(CODE_PATH, "model.h5"))
    return loaded_model

In [0]:
np.unique(y).shape[0]

9498

In [0]:
np.max(y)

9566

In [0]:
max_features

9567

In [0]:
np.max(all_ratings.categ_id)

9566

In [0]:
temp = all_ratings.groupby('userID').agg({'itemID':'count'}).reset_index()
users = temp[temp['itemID'] >= 10]['userID'].values
max_features = np.unique(all_ratings[all_ratings['userID'].isin(users)]['itemID'].values).shape[0]
print(max_features)

9555


In [0]:
np.unique(X).shape

(9542,)

In [0]:
max_features = all_ratings.categ_id.unique().size + 1
embed_size = 128


def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = SpatialDropout1D(0.3)(x)
    x,state1, state2 = Bidirectional(CuDNNGRU(128, return_sequences=True, return_state=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool, state1, state2])
    conc = BatchNormalization()(conc)
    outp = Dense(max_features, activation="softmax")(conc)

    model = Model(inputs=inp, outputs=outp)

#     opt = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    opt = keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,
                  metrics=['sparse_categorical_accuracy'])

#     model.compile(loss='categorical_crossentropy', optimizer=opt,
#                   metrics=['categorical_crossentropy'])

    return model



callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.01,
    patience=1,
    verbose = 1
    )
]


model = get_model()
model.fit(X, y, batch_size=2048 * 8, epochs=15, verbose=1, validation_split=0.01, shuffle=True, callbacks=callbacks_list)


Train on 5049512 samples, validate on 51006 samples
Epoch 1/15
5049512/5049512 [==============================] - 179s 35us/step - loss: 6.9326 - sparse_categorical_accuracy: 0.0305 - val_loss: 6.8955 - val_sparse_categorical_accuracy: 0.0335
Epoch 2/15
5049512/5049512 [==============================] - 174s 34us/step - loss: 6.8558 - sparse_categorical_accuracy: 0.0340 - val_loss: 6.8924 - val_sparse_categorical_accuracy: 0.0356
Epoch 3/15
5049512/5049512 [==============================] - 174s 34us/step - loss: 6.8402 - sparse_categorical_accuracy: 0.0352 - val_loss: 6.9315 - val_sparse_categorical_accuracy: 0.0347

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999776482582e-07.
Epoch 4/15
3545088/5049512 [====================>.........] - ETA: 51s - loss: 6.8366 - sparse_categorical_accuracy: 0.0349

KeyboardInterrupt: ignored

In [0]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.0001,
    patience=1,
    verbose = 1
    )
]
model.fit(X, y, batch_size=2048 * 8, epochs=15, verbose=1, validation_split=0.01, shuffle=True, callbacks=callbacks_list)

Train on 5049512 samples, validate on 51006 samples
Epoch 1/15
5049512/5049512 [==============================] - 152s 30us/step - loss: 6.3488 - sparse_categorical_accuracy: 0.0517 - val_loss: 6.5940 - val_sparse_categorical_accuracy: 0.0485
Epoch 2/15
5049512/5049512 [==============================] - 152s 30us/step - loss: 6.3488 - sparse_categorical_accuracy: 0.0519 - val_loss: 6.5940 - val_sparse_categorical_accuracy: 0.0485

Epoch 00002: ReduceLROnPlateau reducing learning rate to 9.999999939225291e-13.
Epoch 3/15
5049512/5049512 [==============================] - 151s 30us/step - loss: 6.3491 - sparse_categorical_accuracy: 0.0517 - val_loss: 6.5940 - val_sparse_categorical_accuracy: 0.0485

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999960041973e-17.
Epoch 4/15
4505600/5049512 [=========================>....] - ETA: 16s - loss: 6.3493 - sparse_categorical_accuracy: 0.0518

KeyboardInterrupt: ignored

In [0]:
save_model(model, 'cudnngru_maxpoll_with_lr_048acc_val' , '')

Saved model to disk


In [0]:
#model = lstm129()
#model.load_weights('model128.json')

sequences_test = sequences2use.apply(lambda x: x[-maxlen:])
sequences_test = sequences_test.apply(lambda x: [0 for i in range(maxlen - len(x))] + x)
test_users_in_sequences = sorted(set(sequences_test.index) & set(test_users))
X_test = np.array(sequences_test[test_users_in_sequences].tolist())

In [0]:
%%time
from itertools import chain
batch_size = 2048
n_batches = int(X_test.shape[0]/batch_size) + 1
preds = []

for batch_ind in tqdm.tqdm(range(n_batches)):
    batch = X_test[batch_ind*batch_size: (batch_ind + 1)*batch_size]
    curr_preds = model.predict(batch)
    curr_preds = np.argsort(-curr_preds)[:, :20]
    curr_preds = [[cat_to_element_uid[x] for x in row] for row in curr_preds]
    preds.append([' '.join(map(lambda x: str(x), row)) for row in curr_preds])
    
preds = list(chain(*preds))

100%|██████████| 16/16 [00:25<00:00,  1.29s/it]

CPU times: user 26.1 s, sys: 283 ms, total: 26.3 s
Wall time: 25.8 s


In [0]:
len(test_users_in_sequences)

31461

In [0]:
top_20_videos = [2714, 747, 2639, 3783, 2245, 6127, 10061, 3916,
                 8771, 3336, 7079, 3567, 6195, 4141, 1016,
                 7185, 1004, 9467, 72, 1521]

sam_sub = dict()
for el in test_users_in_sequences:
    sam_sub[el] = top_20_videos

In [0]:
keys = [int(x) for x in sam_sub.keys()]
df = pd.DataFrame(keys,columns=['userID'])
df['itemID'] = df.apply(lambda x: sam_sub[x[0]],axis=1)

In [0]:
df.index = df.userID
df.itemID[test_users_in_sequences] = preds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
ans = {}
for el, el1 in df.values:
    try:
        el1 = el1.split(' ')
        for i in range(len(el1)):
            el1[i] = int(el1[i])
        ans[str(el)]= el1
    except:
        ans[str(el)] = el1
    
with open(os.path.join(DATA_PATH, 'nn_048_val_acc.json'), 'w') as f:
    json.dump(ans, f)

In [0]:
# Предсказывает ли сетка одинаковые фильмы для одного и того же пользователя?
with open(os.path.join(DATA_PATH, 'nn_048_val_acc.json'), 'r') as f:
    data = json.load(f)
for key, i in data.items():
  if len(np.unique(i)) < len(i):
    print('Предсказывает одинаковые')
    print(i)
    break

In [0]:
# основная функция для валидации модели
def validation(dict_, file_name):
  d1 = {}
  with open(os.path.join(DATA_PATH, file_name), 'r') as f:
    d1 = json.load(f)
  av = []


  for key in d1.keys():
      if key in dict_.keys():
          ar1 = np.array(d1[str(key)])
         # print(ar1)
          ar2 = np.array(dict_[str(key)])
          inter = np.intersect1d(ar1,ar2)
          av.append(len(inter))
          to_add = []
          for el in ar1:
              if el  in inter:
                  to_add.append(el)
          for el in ar1:
              if el  not in to_add:
                  to_add.append(el)


          to_add = np.array(to_add)
  return np.mean(av),np.var(av)

def shuffle(dict_):
  d = {}
  d1 = {}
  with open(DATA_PATH+'my_best.json', 'r') as f:
    d1 = json.load(f)
  
  for key in dict_.keys():
    if key in d1.keys():
      ar1 = np.array(dict_[key])
       # print(ar1)
      ar2 = np.array(d1[key])
      inter = np.intersect1d(ar1,ar2)

      to_add = []
      for el in ar1:
        if el  in inter:
          to_add.append(el)
      for el in ar1:
        if el  not in to_add:
          to_add.append(el)
        
        
      to_add = np.array(to_add)
        
      d[key] = list(map(int,list(to_add)))

    else:
      d[key] = dict_[key]
  for key in d1.keys():
    if key not in d.keys():
      d[key] = d1[key]
      
  return d
       


In [0]:
validation(data, "blend.json")

(8.428907466861629, 6.130193316686355)